In [346]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

from pycaret.regression import *

In [347]:
root = os.path.join('..', 'data', 'upsampled2')
folder = 'mice_knn_5_upsampled'
df = pd.read_csv(os.path.join(root, f'{folder}_df.csv'))

In [348]:
df['date'] = pd.to_datetime(df['date'])

## Load Model (optional)

In [349]:
# pipeline_path = '../pipelines/base_models/'
# if not os.path.exists(pipeline_path):
#     print("Path to pipeline does not exist")
# else:
#     model_names_to_load = ['br', 'lasso', 'llar', 'en', 'huber']

#     created_models = {}

#     for name in model_names_to_load:
#         created_models[name] = load_model(os.path.join(pipeline_path, name)).named_steps['trained_model']
    
#     print(created_models)

## Pycaret Setup

In [350]:
fh = 60
split_point = df.shape[0] - fh

train_data = df.iloc[:split_point]
test_data = df.iloc[split_point:]

In [351]:
setup(
    data=train_data,
    target='total_number_of_patients',
    test_data=test_data,
    fold_strategy='timeseries',
    data_split_shuffle=False,
    fold_shuffle=False,
    session_id=123
    )

,Description,Value
0,Session id,123
1,Target,total_number_of_patients
2,Target type,Regression
3,Original data shape,"(618, 21)"
4,Transformed data shape,"(618, 23)"
5,Transformed train set shape,"(558, 23)"
6,Transformed test set shape,"(60, 23)"
7,Numeric features,19
8,Date features,1
9,Preprocess,True


## Compare and Analyze Model

In [352]:
compare_models(exclude=['lightgbm', 'par', 'dummy', 'lar'], errors='raise')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
huber,Huber Regressor,3.9044,38.5960,5.8789,0.8993,0.2801,0.1340,0.0130
en,Elastic Net,5.1865,53.9918,6.8726,0.8662,0.3706,0.1690,0.0100
lasso,Lasso Regression,5.1816,54.1577,6.8790,0.8655,0.3748,0.1705,0.0110
llar,Lasso Least Angle Regression,5.1816,54.1587,6.8790,0.8655,0.3748,0.1705,0.0100
br,Bayesian Ridge,5.3641,55.7958,7.0751,0.8611,0.3772,0.1758,0.0110
ridge,Ridge Regression,5.6531,61.4366,7.2679,0.8482,0.3726,0.1785,0.0090
lr,Linear Regression,6.0474,71.7384,7.5854,0.8284,0.3688,0.1847,0.0110
et,Extra Trees Regressor,6.1731,97.1368,9.0756,0.7724,0.3797,0.1865,0.0400
knn,K Neighbors Regressor,6.9888,103.1106,9.6285,0.7521,0.4714,0.2200,0.0100
catboost,CatBoost Regressor,6.5258,106.3597,9.4919,0.7437,0.3920,0.1959,0.3380


HuberRegressor()

In [353]:
model_names_to_create = ['br', 'lasso', 'llar', 'en', 'huber']
created_models = {}

for model_name in model_names_to_create:
    model_object = create_model(model_name, verbose=False)
    created_models[model_name] = model_object

### Bayesian Ridge

In [354]:
evaluate_model(created_models['br'])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### Lasso Regression

In [355]:
evaluate_model(created_models['lasso'])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### Lasso Least Angle Regression

In [356]:
evaluate_model(created_models['llar'])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### Elastic Net

In [357]:
evaluate_model(created_models['en'])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

### Huber Regressor

In [358]:
evaluate_model(created_models['huber'])

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

## Custom Metrics

In [359]:
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

def r2_rounded(y_true, y_pred):
    """Calculates R2 score after rounding predictions to the nearest whole number."""
    return r2_score(y_true, np.round(y_pred))

def rmse_rounded(y_true, y_pred):
    """Calculates RMSE after rounding predictions to the nearest whole number."""
    return np.sqrt(mean_squared_error(y_true, np.round(y_pred)))

def r2_ceil(y_true, y_pred):
    """Calculates R2 score after ceiling predictions to the nearest whole number."""
    return r2_score(y_true, np.ceil(y_pred))

def rmse_ceil(y_true, y_pred):
    """Calculates RMSE after ceiling predictions to the nearest whole number."""
    return np.sqrt(mean_squared_error(y_true, np.ceil(y_pred)))

def mae_rounded(y_true, y_pred):
    """Calculates MAE after rounding predictions to the nearest whole number."""
    return mean_absolute_error(y_true, np.round(y_pred))

def mae_ceil(y_true, y_pred):
    """Calculates MAE after ceiling predictions to the nearest whole number."""
    return mean_absolute_error(y_true, np.ceil(y_pred))

In [360]:
add_metric('R2_Rounded', 'R2_RND', r2_rounded, greater_is_better=True)
add_metric('RMSE_Rounded', 'RMSE_RND', rmse_rounded, greater_is_better=False)
add_metric('MAE_Rounded', 'MAE_RND', mae_rounded, greater_is_better=False)
add_metric('R2_Ceil', 'R2_CEIL', r2_ceil, greater_is_better=True)
add_metric('RMSE_Ceil', 'RMSE_CEIL', rmse_ceil, greater_is_better=False)
add_metric('MAE_Ceil', 'MAE_CEIL', mae_ceil, greater_is_better=False);

## Prediction

In [361]:
holdout_predictions_metric = {}

for model_name, model_object in created_models.items():
    predict_model(model_object, verbose=False)
    holdout_predictions_metric[model_name] = pull()

In [362]:
# pipeline_path = '../pipelines/base_models/'
# os.makedirs(pipeline_path, exist_ok=True)

# for model_name, model_object in created_models.items():
#     save_model(model_object, os.path.join(pipeline_path, model_name));

## Tuning

In [363]:
# param_grids = {
#     'br': {
#         'max_iter': np.arange(50, 301, 50),
#         'alpha_1': np.logspace(-8, -3, 6),
#         'lambda_1': np.logspace(-8, -3, 6),
#         'alpha_2': np.logspace(-8, -3, 6),
#         'lambda_2': np.logspace(-8, -3, 6)
#     },
#     'lasso': {
#         'alpha': np.logspace(-4, 1, 6),
#         'selection': ['cyclic', 'random']
#     },
#     'llar': {
#         'alpha': np.logspace(-4, 1, 6),
#         'jitter': list(np.logspace(-7, -4, 4)) + [None]
#     },
#     'en': {
#         'alpha': np.logspace(-4, 1, 6),
#         'l1_ratio': np.linspace(0, 1, 9),
#         'max_iter': np.arange(100, 1001, 100),
#         'selection': ['cyclic', 'random']
#     },
#     'huber': {
#         'epsilon': [1.0, 1.1, 1.2, 1.25, 1.3, 1.35, 1.4, 1.5, 2.0],
#         'max_iter': np.arange(10, 101, 10),
#         'alpha': np.logspace(-6, -2, 5)
#     }
# }

In [364]:
tuned_models = {}

for model_name, model_object in created_models.items():
    print(f"Tuning model: {model_name}")
    tuned_models[model_name] = tune_model(
        model_object,
        search_library='scikit-optimize',
        n_iter=20,
        early_stopping=True
    )
    print("\r")

Tuning model: br


,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2_RND,RMSE_RND,MAE_RND,R2_CEIL,RMSE_CEIL,MAE_CEIL
Fold,,,,,,,,,,,,
0,8.9612,162.3096,12.7401,0.6889,0.8928,0.2362,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
1,6.5894,73.0020,8.5441,0.8793,0.6016,0.1265,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
2,4.3059,35.8886,5.9907,0.8855,0.2440,0.1895,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
3,6.5629,62.0502,7.8772,0.7921,0.5585,0.3182,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
4,4.2483,25.4208,5.0419,0.9099,0.5591,0.1956,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
5,3.9578,33.7021,5.8054,0.8345,0.2273,0.1817,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
6,2.8874,13.8694,3.7242,0.9321,0.1279,0.1180,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
7,5.9054,73.9223,8.5978,0.8583,0.1981,0.1451,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
8,5.1951,38.8024,6.2292,0.9175,0.2039,0.1350,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for eac

,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2_RND,RMSE_RND,MAE_RND,R2_CEIL,RMSE_CEIL,MAE_CEIL
Fold,,,,,,,,,,,,
0,9.4726,185.8366,13.6322,0.6438,0.9061,0.2443,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
1,5.9954,62.0817,7.8792,0.8973,0.6025,0.1121,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
2,4.1459,31.6954,5.6299,0.8989,0.2316,0.1787,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
3,6.2708,56.7045,7.5302,0.8100,0.5611,0.3035,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
4,4.5803,27.5590,5.2497,0.9023,0.6033,0.2165,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
5,4.0954,29.7627,5.4555,0.8539,0.2631,0.1848,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
6,3.4681,19.7412,4.4431,0.9034,0.1690,0.1626,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
7,4.6591,50.7365,7.1230,0.9027,0.1664,0.1262,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
8,4.3659,29.4630,5.4280,0.9373,0.1717,0.1198,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for eac

...

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for eac

,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2_RND,RMSE_RND,MAE_RND,R2_CEIL,RMSE_CEIL,MAE_CEIL
Fold,,,,,,,,,,,,
0,9.7367,187.9977,13.7112,0.6397,0.8975,0.2418,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
1,5.4577,51.7097,7.1909,0.9145,0.5963,0.1036,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
2,4.2058,35.1721,5.9306,0.8878,0.2420,0.1873,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
3,6.4468,60.2131,7.7597,0.7983,0.5497,0.3102,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
4,4.2385,24.8063,4.9806,0.9121,0.5638,0.1951,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
5,3.7233,27.4756,5.2417,0.8651,0.2032,0.1622,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
6,2.9323,14.5415,3.8133,0.9288,0.1294,0.1204,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
7,5.5344,67.0775,8.1901,0.8714,0.1851,0.1360,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
8,4.8278,34.5427,5.8773,0.9265,0.1910,0.1256,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for eac

,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2_RND,RMSE_RND,MAE_RND,R2_CEIL,RMSE_CEIL,MAE_CEIL
Fold,,,,,,,,,,,,
0,6.8552,111.9973,10.5829,0.7853,0.7793,0.1839,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
1,2.9561,16.1307,4.0163,0.9733,0.4687,0.0637,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
2,2.9165,19.5300,4.4193,0.9377,0.2438,0.1265,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
3,3.1161,20.7385,4.5540,0.9305,0.2324,0.1341,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
4,2.6006,13.2180,3.6357,0.9532,0.2031,0.1288,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
5,4.0421,36.0191,6.0016,0.8231,0.2478,0.1870,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
6,2.6361,15.5708,3.9460,0.9238,0.1164,0.1041,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
7,4.1175,51.8153,7.1983,0.9007,0.1667,0.1112,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000
8,3.5657,29.8929,5.4674,0.9364,0.2025,0.1097,0.0000,-0.0000,-0.0000,0.0000,-0.0000,-0.0000


Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for eac

In [365]:
# pipeline_path = '../pipelines/tuned_models/'
# if not os.path.exists(pipeline_path):
#     print("Path to pipeline does not exist")
# else:
#     model_names_to_load = ['br', 'lasso', 'llar', 'en', 'huber']

#     tuned_models = {}

#     for name in model_names_to_load:
#         tuned_models[name] = load_model(os.path.join(pipeline_path, name)).named_steps['trained_model']
    
#     print(tuned_models)

In [366]:
# pipeline_path = '../pipelines/tuned_models/'
# os.makedirs(pipeline_path, exist_ok=True)

# for model_name, model_object in tuned_models.items():
#     save_model(model_object, os.path.join(pipeline_path, model_name));

In [367]:
tuning_predictions_metric = {}

for model_name, model_object in tuned_models.items():
    predict_model(model_object, verbose=False)
    tuning_predictions_metric[model_name] = pull()

In [368]:
holdout_predictions_metric['br']

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,R2_RND,RMSE_RND,MAE_RND,R2_CEIL,RMSE_CEIL,MAE_CEIL
0,Bayesian Ridge,4.0049,27.0453,5.2005,0.9357,0.1036,0.0741,0.9363,5.1785,3.95,0.9371,5.1462,3.9167


In [369]:
list_of_metric_dfs = []

file_name = 'temp_excel.xlsx'

# Iterate through the dictionary and append each DataFrame to the list
for model_name, metrics_df in holdout_predictions_metric.items():
    list_of_metric_dfs.append(metrics_df)

# Concatenate all DataFrames into a single one
print("--- Holdout Predictions DataFrame ---", end='\n\n')
results_df = pd.concat(list_of_metric_dfs, ignore_index=True)
print(results_df.to_string())

with pd.ExcelWriter(file_name) as writer:
    results_df.to_excel(writer, sheet_name='Base Model Metrics', index=False)

list_of_metric_dfs = []
for model_name, metrics_df in tuning_predictions_metric.items():
    list_of_metric_dfs.append(metrics_df)

print("\n--- Tuned Predictions DataFrame ---", end='\n\n')
results_df = pd.concat(list_of_metric_dfs, ignore_index=True)
print(results_df.to_string())

with pd.ExcelWriter(file_name, mode='a') as writer:
    results_df.to_excel(writer, sheet_name='Tuned Model Metrics', index=False)

--- Holdout Predictions DataFrame ---

                          Model     MAE      MSE    RMSE      R2   RMSLE    MAPE  R2_RND  RMSE_RND  MAE_RND  R2_CEIL  RMSE_CEIL  MAE_CEIL
0                Bayesian Ridge  4.0049  27.0453  5.2005  0.9357  0.1036  0.0741  0.9363    5.1785   3.9500   0.9371     5.1462    3.9167
1              Lasso Regression  4.1728  30.6580  5.5370  0.9272  0.1086  0.0777  0.9268    5.5513   4.1500   0.9272     5.5362    4.1500
2  Lasso Least Angle Regression  4.1731  30.6613  5.5373  0.9272  0.1086  0.0777  0.9268    5.5513   4.1500   0.9272     5.5362    4.1500
3                   Elastic Net  4.0589  28.5260  5.3410  0.9322  0.1063  0.0762  0.9313    5.3759   4.0667   0.9323     5.3385    4.0333
4               Huber Regressor  4.3796  54.9338  7.4117  0.8695  0.1142  0.0723  0.8695    7.4117   4.3667   0.8643     7.5576    4.4500

--- Tuned Predictions DataFrame ---

                          Model     MAE      MSE    RMSE      R2   RMSLE    MAPE  R2_RND  RMSE_